In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# Testing an <span style="font-variant:small-caps;">Antlr</span> Grammar via `grun` or `pygrun`

Our grammar is stored in the file `Expr.g4`.  In order to inspect it, we use the command line tool `cat`.  This will work with MacOs and Linux.  On Windows,
either use the power shell, which understands `cat`,  or us the command `type` instead.  The option  `-n` of `cat` provides numbered output.

In [ ]:
!cat -n Expr.g4

Note that this grammar does not contain any embedded actions.  Hence we cannot compute anything with it.  We will only be able to 
check whether a given string is generated by this grammar.  Provided you have stored the file `antlr-4.8-complete.jar` in the directory
`/usr/local/lib/` we can generate both the scanner and the parser using the following command:

In [ ]:
!java -jar /usr/local/lib/antlr-4.8-complete.jar -Dlanguage=Python3 Expr.g4

On Unix-like systems it is also convenient to define an <em style="color:blue">alias</em> of the following form in the file `.bashrc`
that is used to initialize the bash shell:
```
alias antlr='java -jar /usr/local/lib/antlr-4.8-complete.jar -Dlanguage=Python3' 
```
We can use the system command `ls` to see which files have been generated by <span style="font-variant:small-caps;">Antlr</span>.
If you are using a windows system you have to use the command `dir` instead.

In [ ]:
!ls -l

The files `ExprLexer.py` and `ExprParser.py` contain the generated scanner and parser, respectively.
If we want to test the parser in this notebook, we have to import these files.

In [ ]:
from ExprLexer  import ExprLexer
from ExprParser import ExprParser

In order to use the <span style="font-variant:small-caps;">Antlr</span> runtime, we have to import it.
Note that this runtime is not contained in `antlr-4.8-complete.jar` and needs to be installed separately using the following command:
```
conda install -c conda-forge antlr
```

In [ ]:
import antlr4

Now we can parse a string.  The function `parser_string` takes the string `s` as its argument and checks, whether this string
can be parsed as an arithmetic expression.

In [ ]:
def parse_string(string): 
    inputStream = antlr4.InputStream(string)
    lexer       = ExprLexer(inputStream)
    stream      = antlr4.CommonTokenStream(lexer)
    parser      = ExprParser(stream)
    parser.start()

In [ ]:
parse_string('1 + 2 * 3 - 4')

As there is no syntax error, the string `'1 + 2 * 3 - 4'` adheres to the specification given by our grammar.
Lets try a string that is not generated by our grammar.

In [ ]:
parse_string('1 + 2 * 3 ** 4')

As the operator `**` is not supported by our grammar, we get a syntax error at the last occurrence of the character `*` in the given string.

We can also generate a parse tree with our grammar.  However, for this to work <span style="font-variant:small-caps;">Antlr</span>
first has to generate a `java` parser.  Hence we have to call `antlr4` again, but this time with `Java` as the target language.

In [ ]:
!java -jar /usr/local/lib/antlr-4.8-complete.jar -Dlanguage=Java Expr.g4

This command has generated some `java` files for us that contain a both a lexer and a parser.  However, this time these are `.java`-files.

In [ ]:
!ls *.java

We have to compile the generated `java` files.  You might have to change the path to `antlr-4.8-complete.jar`.

In [ ]:
!javac -cp .:/usr/local/lib/antlr-4.8-complete.jar *.java

Next, we can start the so called `TestRig` to generate and display the <em style="color:blue">parse tree</em> for a given string.

In [ ]:
!echo "1+2*3-4" | java -cp .:/usr/local/lib/antlr-4.8-complete.jar org.antlr.v4.gui.TestRig Expr start -gui

There is a python equivalent to this `java` program that is called `pygrun`, which is short for <u>py</u>thon <u>g</u>rammar  <u>run</u>. 
However, it only provides the `--tree` option.  Furthermore, in order for `pygrun` to find the `antlr4` runtime, we have to set the environment variable `PYTHONPATH`.

In [ ]:
!echo "1 + 2 * 3 - 4" | PYTHONPATH="/Users/stroetmann/opt/anaconda3/pkgs/antlr-python-runtime-4.8-py38h32f6830_1/lib/python3.8/site-packages" ../pygrun Expr start --tree 

In [ ]:
!rm *.py *.tokens *.interp